- Code written by Jamal
- Annotated by Aidan
- 1 of 3 Python scripts corresponding to "Toward Fairness in Face Matching Algorithms"

In [ ]:
# # I think this is looking at files from a file tree and identifying duplicates ???

# from os import listdir
# import shutil
# from os.path import isfile, join

# # root directory

# img_path = "/home/jamal/Desktop/LFW attributes/lfw/"

# # output directory 
# save_path = "/home/jamal/Desktop/LFW attributes/more_than_once/"

# onlyfiles = [f for f in listdir(img_path)]
# 
# for i in range(len(onlyfiles)):
#     tem_path = img_path + onlyfiles[i]
#     if len(listdir(tem_path))>1:
#         s_path = save_path + onlyfiles[i]
#         shutil.copytree(tem_path,s_path)
#     print(i/len(onlyfiles))

In [4]:
# IMPORTING LIBARIES #

# tabular data manipulation
import pandas as pd

# array manipulation
import numpy as np

# visualizations
import matplotlib.pyplot as plt

# operating system status
import os

# for resizing images
from skimage import io, transform

# library for adam optimizer
import torch.optim as optim

# DataLoader is a Python iterable over a dataset
from torch.utils.data import Dataset, DataLoader

# for grabbing a random set of images
from torch.utils.data.sampler import SubsetRandomSampler

# for confusion matrix (model evaluation)
import sklearn.metrics as mt

In [ ]:
# additional files is located in /home/jamal/Downloads/additional_files/

# tar file with main NN progress saves
def save_checkpoint(state, filename='/home/jamal/Downloads/checkpoint_0.001.pth.tar'):
    torch.save(state, filename)
# tar file with adversarial NN progress saves
def save_checkpoint_A(state, filename='/home/jamal/Downloads/checkpoint_A_0.001.pth.tar'):
    torch.save(state, filename)

# path to csv    
file_path ='/home/jamal/Downloads/umdfaces_batch1_ultraface_new1.csv'

# path to images
images_path = '/home/jamal/Downloads/umdfaces_batch1/'

# used for before and during/after a model is trained
# main  = 0, testing = 0
load_net = 0

# attributes/columns for celebrity data set
# columns = ['ImgId','5_o_Clock_Shadow', ' Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']

# attributes/columns for UMD data set
columns=['SUBJECT_ID','FILE','FACE_X','FACE_Y','FACE_H','FACE_W','PR_MALE','PR_FEMALE']

# read celebrity csv
cele_attrib = pd.read_csv(file_path,delimiter = ",",names = columns)
# # commented out setting the index according to the subject id
# lfw = cele_attrib.set_index('SUBJECT_ID')
# lfw = cele_attrib

In [ ]:
# # just visualizing random images from the celebrity data set using matplotlib
# len_attrib = len(cele_attrib)
# Select random images form celeba dataset
# rnd_set = np.random.permutation(len_attrib)[0:5]
# for i in rnd_set:
#     idx = ("{:06d}.png".format(i))
#     img_path = images_path+idx
#     img = plt.imread(img_path)
#     plt.imshow(img)
#     plt.show()
#     print(cele_attrib['Male'][i-1])

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn

# main neural network with both image inputs
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()

        # it seems like the size of the layers are different here than in the final paper (could be that Jamal used different py script)

        # for large image (150 by 150) with 3 convulational/max-pooling layers 
        self.conv11 = nn.Conv2d(3,64,5)
        self.n11 = nn.BatchNorm2d(64)
        self.conv21 = nn.Conv2d(64,96,5)
        self.n21 = nn.BatchNorm2d(96)
        self.conv31 = nn.Conv2d(96,128,3)
        self.n31 = nn.BatchNorm2d(128)

        # for small image (12 by 12) with 2 convulational/max-pooling layers
        self.conv12 = nn.Conv2d(3,32,7)
        self.n12 = nn.BatchNorm2d(32)
        self.conv22 = nn.Conv2d(32,75,3)
        self.n22 = nn.BatchNorm2d(75)
        #self.conv32 = nn.Conv2d(75,100,3)
        #self.n32 = nn.BatchNorm2d(100)
        
        # flattening occurs here

        # for determining if the same face
        self.ip1 = nn.Linear(1712,128)   # change the first parameter in case you change the size of the small image
        self.ip2 = nn.Linear(128,
                             # output corresponds to two nodes: one for same face and the other for different face? 
                             2)

        # seems like this would be for the adversarial NN ? I think this code is actually not used for the main NN based on forward(self,x,y)
        self.ip3 = nn.Linear(1712,128,
                             # bias set to False, the layer will not learn an additive bias (Default: True)
                             False)   # change the first parameter in case you change the size of the small image
        self.ip4 = nn.Linear(128,
                             # output corresponds to two nodes: one for female and the other for male?       
                             2,
                             # bias set to False, the layer will not learn an additive bias (Default: True)
                             False)
        
    
    def forward(self,x,y):
        # large image
        x = self.conv11(x)
        x = self.n11(x)
        x = F.relu(x)
        # maxpooling
        x = F.max_pool2d(x,5,stride = 3)
        
        x = self.conv21(x)
        x = self.n21(x)
        x = F.relu(x)
        # maxpooling
        x = F.max_pool2d(x,5,stride = 3)

        x = self.conv31(x)
        x = self.n31(x)
        x = F.relu(x)
        # maxpooling
        x = F.max_pool2d(x,5)
        # flattening large image
        x = x.view(-1,512)
        
        # small image
        y = self.conv12(y)
        
        y = self.n12(y)
        y = F.relu(y)
        # y = F.max_pool2d(y,5,stride = 1)
        y = self.conv22(y)
        y = self.n22(y)
        y = F.relu(y)
        # y = F.max_pool2d(y,5,stride = 2)
                
        # seems like Jamal originally tried this with three layers
        # y = self.conv32(y)
        # y = F.max_pool2d(y,3)
        # y = self.n32(y)
        # y = F.relu(y)

        y = y.view(-1,1200)  # change the second parameter in case you change the size of the small image
        
        # FLATTENING! aka concatenating ?
        x = torch.cat((x,y),1)
        x1 = self.ip1(x)
        x1 = F.relu(x1)
        x1 = self.ip2(x1)

        # x2 represents the large image in the adversarial network
        # x2 = self.ip3(x)
        # x2 = F.relu(x2)
        # x2 = self.ip4(x2)
        # x2 = x2.mul(-1)
        # x = F.relu(x)
        # x = F.softmax(x,1) # why remove softmax?
        return x1, x
      

# adversarial neural network (male/female identifier)
class Net_A(nn.Module):
    
    def __init__(self):
        super(Net_A,self).__init__()
        
        self.ip3 = nn.Linear(
            # input = size of flattened image
            1712,
            # output (heavily condensing)
            128,
            # bias set to False, the layer will not learn an additive bias (Default: True)
            False)   # change the first parameter in case you change the size of the small image
        
        # Relu between layers    
        
        self.ip4 = nn.Linear(128,
                             # output corresponds to two nodes: one for female and the other for male ?
                             2,
                             # bias set to False, the layer will not learn an additive bias (Default: True)
                             False)
        
    
    def forward(self,x):
        # takes only the larger image as an input to test in the model
        x2 = self.ip3(x)
        x2 = F.relu(x2)
        x2 = self.ip4(x2)
        # x2 = x2.mul(-1)
        # x = F.relu(x)
        # x = F.softmax(x,1)
        return x2

In [ ]:
# for resizing images
class Male_Female_dataset(Dataset):

    def __init__(self,root_dir,shape,transform=None):
        # directory of csv
        self.root_dir = root_dir
        # no transformations
        self.transform = transform
        # shape of image
        self.shape = shape
        # directory of images 
        self.images_names = os.listdir(images_path)
        
    def __len__(self):
        # get the number of celebrity attributes  
        return len(cele_attrib)
        
    def __getitem__(self,idx):
        # tensor of 1 random representing random uniform scaler => balanced data set
        # 50% likelyhood of creating a low resolution version of the same person 
        # 50% likelyhood of creating a low resolution version of a different person 
        t = torch.rand(1);
        # convert row (image) at a given index (argument/idx) to list 
        lst = cele_attrib.loc[idx].tolist()
        # get image name
        img1_name = os.path.join(self.root_dir,lst[1])
        # input reading of image according to name
        image1 = io.imread(img1_name)
        # ['SUBJECT_ID','FILE','FACE_X','FACE_Y','FACE_H','FACE_W','PR_MALE','PR_FEMALE'] ??? I really am not sure about this one?
        # Face Y Position + Face Width ?
        # Face X Position + Face Height ?
        image1 = image1[int(lst[3]):int(lst[3]+lst[5]),int(lst[2]):int(lst[2]+lst[4]),:]
        # import cv2
        # cv2.imshow('a',img)
        # cv2.waitKey(1000)
        # cv2.destroyAllWindows()
        
        # resizing to 150 by 150
        image1 = transform.resize(image1,(150,150))
       
        # 50% likelyhood of creating a low resolution version of a different person
        if t > 0.5: #different
            different = False
            while not(different):
                h = torch.randint(0,len(self),(1,1))
                ht = torch.Tensor.numpy(h)
                if lfw.loc[idx][0] != lfw.loc[int(ht[0])][0]:
                    different = True
            
            lstd = cele_attrib.loc[int(ht[0])].tolist()        
                
            img2_name = os.path.join(self.root_dir,lstd[1])
            image2 = io.imread(img2_name)
            image2 = image2[int(lstd[3]):int(lstd[3]+lstd[5]),int(lstd[2]):int(lstd[2]+lstd[4]),:]
            # import cv2
            # cv2.imshow('a',image2)
            # cv2.waitKey(1000)
            # cv2.destroyAllWindows()
            image2 = transform.resize(image2,self.shape)
            image2 = torch.Tensor.float(torch.from_numpy(image2))
            image2 = (torch.Tensor.permute(image2,(2,0,1)))
            image2 = image2/256.0
            image1 = torch.Tensor.float(torch.from_numpy(image1))
            image1 = (torch.Tensor.permute(image1,(2,0,1)))
            image1 = image1/256.0
            annot = 0
            gender =  1 if lst[-2]>.5 else -1

        # 50% likelyhood of creating a low resolution version of the same person     
        else: 
            id = cele_attrib.loc[idx].tolist()[0]
            idx_new = idx+1
            if  cele_attrib.loc[idx+1].tolist()[0]==id:
                lstd = cele_attrib.loc[idx+1].tolist()
                sta = os.path.join(self.root_dir,lstd[1])
            else:
                lstd = cele_attrib.loc[idx-1].tolist()
                sta = os.path.join(self.root_dir,lstd[1])
            # st = img1_name.split('/')
            # st = st[-1]
            # st = st.split('_')
            # nn = st[0:-1]
            # name = ' '.join(nn)
            # num = int(st[-1].split('.')[0])
            
            # cc = lfw.loc[name]
            # if isinstance(cc.imagenum,str):
            #     print(cc)
            # ls = cc.imagenum.tolist()
            # if str(num) in ls:
            #     ls.remove(str(num))
            # 
            # h = torch.randint(0,len(ls),(1,1))
            # ht = torch.Tensor.numpy(h)
            # ht = ht[0]
            # num2 = str(int(ls[int(ht)])).zfill(4)
            # sta = img1_name.split('_')
            # stemp = sta[-1].split('.')
            # stemp[0] = num2
            # stemp = '.'.join(stemp)
            # sta[-1]=stemp
            # sta = '_'.join(sta)
            image2 = io.imread(sta)
            image2 = transform.resize(image2,self.shape)
            image2 = torch.Tensor.float(torch.from_numpy(image2))
            image2 = (torch.Tensor.permute(image2,(2,0,1)))
            image2 = image2/256.0
            image1 = torch.Tensor.float(torch.from_numpy(image1))
            image1 = (torch.Tensor.permute(image1,(2,0,1)))
            image1 = image1/256.0
            annot = 1
            gender = 1 if lst[-2]>.5 else -1
        # annot = annot.astype('float').reshape(-1,2)
        sample = {'image1':image1,'image2':image2, 'same' : annot, 'gender' : gender}
        return sample

## Execution Script

In [ ]:
# use_cuda is true if cuda is available to use for GPU capabilities
use_cuda = torch.cuda.is_available()
# use_cuda=0
# if cude is not available, use CPU
device = torch.device("cuda" if use_cuda else "cpu")
# prints whether the processing devise is a CPU or cuda GPU
print(device)
# shape of smaller image
shape = (12,12)
# shape of larger image
ip_shape = (150,150)
# 80% of data will be used for training
train_ratio = 0.8
# remaining 20% of data will be used for validation (most)
val_ratio = 0.2
# for actual test, training and validation ratio will both be zero and test ratio will be 1
test_ratio = 1-train_ratio-val_ratio
# create data set object!
dataset = Male_Female_dataset(images_path,shape)
# dataset.__getitem__(175534-1)

# if load_net == 1 (loading) -> testing
if load_net:
    # run both nets on either cpu or gpu (CUDA)
    net = Net().to(device)
    net_A = Net_A().to(device)

    # NOTE: checkpoints save epoch #, state, and optimizer

    # loading NN from last checkpoint where checkpoint 
    checkpoint = torch.load('/home/jamal/Downloads/additional_files/checkpoint_0.001.pth.tar')
    # loading adversarial NN from last checkpoint where 
    checkpoint_A = torch.load('/home/jamal/Downloads/additional_files/checkpoint_A_0.001.pth.tar')

    # loading main NN checkpoint where checkpoint 
    net.load_state_dict ( checkpoint['state_dict'])
    # using adam optimizer for backpropagation
    optimizer = optim.Adam(net.parameters(),
                           # learning rate of 0.001
                           lr = 0.001, 
                           # learning rate decay of .0005
                           weight_decay = 0.0005)
    optimizer.load_state_dict = checkpoint['optimizer']
    
    # loading adversarial NN checkpoint state
    net_A.load_state_dict ( checkpoint_A['state_dict'])
    # using adam optimizer for backpropagation
    optimizer_A = optim.Adam(net_A.parameters(),
                             # learning rate of 0.001
                             lr = 0.001,
                             # learning rate decay of .0005
                             weight_decay = 0.0005)
    # loading adversarial NN checkpoint optizer
    optimizer_A.load_state_dict = checkpoint_A['optimizer']

# if load_net == 0 (not loading) -> development
else:
    # instantiating main NN to be run on cpu or gpu
    net = Net().to(device)
    # instantiating adversarial NN
    net_A = Net_A().to(device)
    # adam optimizer, .001 learning rate, and .0005 weight decay for backpropagation of both NN
    optimizer = optim.Adam(net.parameters(),lr = 0.001, weight_decay = 0.0005)
    optimizer_A = optim.Adam(net_A.parameters(),lr = 0.001, weight_decay = 0.0005)
    # setting checkpoint epochs to zero
    checkpoint= {'epoch':0}
    
# optimizer = optim.Adam(net.parameters(),lr = 0.001, weight_decay = 0.0005)
# optimizer_A = optim.Adam(net.parameters(),lr = 0.001, weight_decay = 0.0005)

# create randomize numpy array with index numbers ranging from 0 to 5 less than the size of the data set
# not sure why it is -5?
index = np.random.permutation(len(dataset)-5)
# number of entries in the training data set
train_data_length = int(train_ratio*len(index))
# number of entries in the validation data set
val_data_length = int(val_ratio*len(index))
# number of entries in the testing data set
test_data_length = int(test_ratio*len(index))
# train data indexes correspond to index 0 of index array to train data length 
train_index = index[:train_data_length]
# validation data indexes correspond to the index values of train data length to validation data length   
val_index = index[train_data_length:(train_data_length+val_data_length)]
# test data indexes correspond to the index values of  validation data length to test data length   
test_index = index[train_data_length+val_data_length:]

# load data 
train_dataloader = DataLoader( # more or less a series of image paths
                              dataset,
                              # number of samples per load
                              batch_size=100,
                              # numpy array generated a couple lines above is used as method for sampling from dataset
                              sampler = SubsetRandomSampler(train_index))

# error of face matching
err_same = []
# error of gender matching
err_gender = []

# accuracy of face matching
acc1 = []
# accuracy of gender matching
acc2 = []

# minimization criterion of face matching NN
criterion = nn.CrossEntropyLoss()
# minimization criterion of gender matching NN
criterion2 = nn.CrossEntropyLoss()

# not sure what t is ? it's only use seems to be commented out
t = -1

# run for 5 epochs
for ep in range(checkpoint['epoch'],5):
    for i,data in enumerate(train_dataloader):
        
        # Zeros descent gradient to help generalize after each batch
        optimizer.zero_grad()
        optimizer_A.zero_grad()
        
        # input1 = high res true image
        # input2 = low res true or false image
        # label = name corresponding to true image path
        # gender = male or female label
        input1, input2 , label, gender = data.items()
        # run on cuda gpu if available
        input1, input2 = input1[1].to(device), input2[1].to(device)
        label1 = label[1].to(device)
        gender = gender[1].to(device)
        # not sure why it is (gender+1)/2? 
        gender = (gender+1)/2
        #label = torch.Tensor.long(label[1])
        
        # feed input images 1 and 2 into neural network
        output = net(input1,input2)
        # feed output of main face matching NN to gender matching NN
        output_A = net_A(output[1])

        # use cross entropy loss as minimization criterion for both main an adv. NN
        # use true image as label
        loss1 = criterion (output[0],label1)
        #loss1.backward(retain_graph=True)
        # optimizer.step()

        # use gender as label
        loss2 = criterion2 (output_A,gender)
        
        # backwards propagation 
        # not sure why retain graph ? In essence, it will retain any necessary information to calculate a certain variable, so that we can do backward pass on it.
        loss2.backward(retain_graph=True)
        # gender matching NN gradient step
        optimizer_A.step()

        # formula: L = Ly - wLd
        loss1 = loss1 - loss2.mul(1)  #new line 
        #loss2 = loss2.abs()
        # t = t*-1

        # backwards propagation 
        loss1.backward()
        # face matching NN gradient step
        optimizer.step()
        # adding loss as an entry to face matching list
        err_same.append(loss1.item())
        # adding loss as an entry to face matching list
        err_gender.append(loss2.item())

        # if 5th iteration, print the loss, iteration #, and epoch for each of the NN
        if (i%5==0):
            print (loss1,i,ep)
            print (loss2,i,ep)
            #print (output[1])

        # if 50th iteration, check accuracy of model          
        if (i%50==0):
            # create a numpy array from the indexes from the validation array
            val_index1 = np.random.permutation(val_index)[:100]
            # test for accuracy using 30 images from validation data 
            val_dataloader = DataLoader(dataset,batch_size=30,sampler = SubsetRandomSampler(val_index1))
            # make validation data iterable
            val_iter = iter(val_dataloader)

            # counter variables
            total = 0
            correct1 = 0
            correct2 = 0
            # literally just going through and checking if the model predicts the true value (same face / which gender) correctly
            for j,dataj in enumerate(val_dataloader):
                input1j, input2j, labelj, gender = dataj.items()
                input1j, input2j = input1j[1].to(device), input2j[1].to(device)
                
                labelj = labelj[1].to(device)
                gender = gender[1].to(device)
                gender = (gender+1)/2
                output = net(input1j,input2j)
                output_A = net_A(output[1])
                _,predicted1 = torch.max(output[0].data,1)
                _,predicted2 = torch.max(output_A.data,1)
                total +=labelj.size(0)
                correct1 += (predicted1 == labelj).sum().item()
                correct2 += (predicted2 == gender).sum().item()
            # prints out percent accurate for both NN
            print('Accuracy_LH: %d %%'%(100*correct1/total))
            print('Accuracy_MF: %d %%'%(100*correct2/total))
            
            # record the percent accurate for both NN into respective lists
            acc1.append(100*correct1/total) #low high
            acc2.append(100*correct2/total) # male female

    # outside of inner loop
    # saves for every epoch

    # save the checkpoint to be loaded for next use
    save_checkpoint({
            # increase the track of the current epoch #
            'epoch': ep + 1,
            # save Python dictionary object that maps each layer to its parameter tensor.
            'state_dict': net.state_dict(),
            # update optimizer conditions (because it may not be zeroed)
            'optimizer' : optimizer.state_dict(),
        })
    save_checkpoint_A({
            'epoch': ep + 1,
            'state_dict': net_A.state_dict(),
            
            'optimizer' : optimizer_A.state_dict(),
        })

In [ ]:
# FOR CELEB DATA SET #
  
# val_index1 = np.random.permutation(val_index)[:100]
# val_dataloader = DataLoader(dataset,batch_size=30)
# val_iter = iter(val_dataloader)
# predicted_lables = []
# real_labels_g = []
# predicted_lables_g = []
# real_labels = []
# total = 0
# correct1 = 0
# correct2 = 0
# for j,dataj in enumerate(val_dataloader):
#     input1j, input2j, labelj, gender = dataj.items()
#     input1j, input2j = input1j[1].to(device), input2j[1].to(device)
#     
#     labelj = labelj[1].to(device)
#     gender = gender[1].to(device)
#     gender = (gender+1)/2
#     output = net(input1j,input2j)
#     _,predicted1 = torch.max(output[0].data,1)
#     _,predicted2 = torch.max(output[1].data,1)
#     total +=labelj.size(0)
#     correct1 += (predicted1 == labelj).sum().item()
#     correct2 += (predicted2 == gender).sum().item()
#     predicted_lables.append(torch.Tensor.numpy(predicted1))
#     real_labels.append(torch.Tensor.numpy(labelj))
#     
#     predicted_lables_g.append(torch.Tensor.numpy(predicted2))
#     real_labels_g.append(torch.Tensor.numpy(gender))
#     print('Accuracy_LH: %d %%'%(100*correct1/total))
#     print('Accuracy_MF: %d %%'%(100*correct2/total))
#     acc1.append(100*correct1/total)
#     acc2.append(100*correct2/total)
#     print(mt.confusion_matrix(np.array(real_labels).flatten(),np.array(predicted_lables).flatten()))
#     
#     print(mt.confusion_matrix(np.array(real_labels_g).flatten(),np.array(predicted_lables_g).flatten()))
# # for ep in range(checkpoint['epoch'],5):
#     for i,data in enumerate(train_dataloader):
#         optimizer.zero_grad()
#         input1, input2 , label = data.items()
#         input1, input2 = input1[1].to(device), input2[1].to(device)
#         label1 = label[1].to(device)
#         #label = torch.Tensor.long(label[1])
#         output = net(input1,input2)
#         loss = criterion (output,label1)
#         loss.backward()
#         optimizer.step()
#         err.append(loss.item())
#         if (i%5==0):
#             print (loss,i,ep)
#         
#         if (i%50==0):
#             val_index1 = np.random.permutation(val_index)[:100]
#             val_dataloader = DataLoader(dataset,batch_size=30,sampler = SubsetRandomSampler(val_index1))
#             val_iter = iter(val_dataloader)
#     
#             total = 0
#             correct = 0
#             for j,dataj in enumerate(val_dataloader):
#                 input1j, input2j, labelj = dataj.items()
#                 input1j, input2j = input1j[1].to(device), input2j[1].to(device)
#                 
#                 labelj = labelj[1].to(device)
#                 output = net(input1j,input2j)
#                 _,predicted = torch.max(output.data,1)
#                 total +=labelj.size(0)
#                 correct += (predicted == labelj).sum().item()
#             print('Accuracy: %d %%'%(100*correct/total))
#             acc.append(100*correct/total)
#     save_checkpoint({
#             'epoch': ep + 1,
#             'state_dict': net.state_dict(),
#             
#             'optimizer' : optimizer.state_dict(),
#         })
#     
# 
#         
#         
#         
# 
#     
# # sample = dataset[1]
# # img = plt.imread(images_path+'000001.png')
# # img2 = np.expand_dims(img,0)
# # img3 = np.rollaxis(img2,3,1)
# # output = Net(torch.from_numpy(img3))